In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/RF_classification.csv', encoding='cp1251', sep=';')
# смотрим данные
data

,age,gender,default
0,70,Мужской,Да
1,64,Мужской,Нет
2,69,Женский,Да
3,68,Мужской,Нет
4,65,Женский,Нет


In [3]:
# создаем массив меток
y = data.pop('default')
# создаем массив признаков
X = pd.get_dummies(data)

In [4]:
# импортируем класс RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
# создаем экземпляр класса RandomForestClassifier
forest = RandomForestClassifier(n_estimators=10, random_state=152, 
                                oob_score=True, n_jobs=-1)
# подгоняем модель
forest.fit(X, y)
# вычисляем правильность по методу OOB
print('Правильность по методу OOB: {:.2f}'.format(forest.oob_score_))

Правильность по методу OOB: 0.40


In [5]:
# прогнозируем классы зависимой переменной по методу OOB
oob_predictions = np.argmax(forest.oob_decision_function_, axis=1)
# преобразуем прогнозы в объект DataFrame
oob_predictions = pd.DataFrame(oob_predictions, index=data.index, columns=['OOB_Predictions'])
# конкатенируем массив признаков, массив меток и OOB прогнозы
oob_results = pd.concat([X, y, oob_predictions], axis=1)
oob_results

,age,gender_Женский,gender_Мужской,default,OOB_Predictions
0,70,0,1,Да,1
1,64,0,1,Нет,1
2,69,1,0,Да,1
3,68,0,1,Нет,0
4,65,1,0,Нет,1


In [6]:
# прогнозируем вероятности классов по методу OOB
print('Вероятности, спрогнозированные по методу OOB:\n{}'.format(
    forest.oob_decision_function_))

Вероятности, спрогнозированные по методу OOB:
[[0.2        0.8       ]
 [0.         1.        ]
 [0.2        0.8       ]
 [1.         0.        ]
 [0.33333333 0.66666667]]


In [7]:
# вычисляем правильность по обычному методу
print('Правильность по обычному методу: {:.3f}'.format(forest.score(X, y)))

Правильность по обычному методу: 1.000


In [8]:
# прогнозируем классы зависимой переменной по обычному методу
predictions = forest.predict(X)
# преобразуем прогнозы в объект DataFrame
predictions = pd.DataFrame(predictions, index=data.index, columns=['Predictions'])
# конкатенируем массив признаков, массив меток и прогнозы
results = pd.concat([X, y, predictions], axis=1)
results

,age,gender_Женский,gender_Мужской,default,Predictions
0,70,0,1,Да,Да
1,64,0,1,Нет,Нет
2,69,1,0,Да,Да
3,68,0,1,Нет,Нет
4,65,1,0,Нет,Нет


In [9]:
# прогнозируем вероятности классов по обычному методу
print('Вероятности, спрогнозированные по обычному методу:\n{}'.format(
    forest.predict_proba(X)))

Вероятности, спрогнозированные по обычному методу:
[[0.6 0.4]
 [0.  1. ]
 [0.6 0.4]
 [0.3 0.7]
 [0.1 0.9]]


In [10]:
# вычисляем листовые вероятности каждого дерева для первого наблюдения 
for tree_in_forest in forest.estimators_:    
    print(tree_in_forest.predict_proba(X)[0])

[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]


In [11]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/RF_regression.csv', encoding='cp1251', sep=';')
# смотрим данные
data

,age,gender,days_of_delinquency
0,70,Мужской,18
1,64,Мужской,15
2,69,Женский,17
3,68,Мужской,19
4,65,Женский,15
5,63,Мужской,16
6,55,Женский,13
7,60,Мужской,14
8,67,Мужской,16
9,68,Женский,14


In [12]:
# создаем массив меток
y = data.pop('days_of_delinquency')
# создаем массив признаков
X = pd.get_dummies(data)

In [13]:
# импортируем класс RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
# создаем экземпляр класса RandomForestRegressor
forest = RandomForestRegressor(n_estimators=10, random_state=152, 
                               oob_score=True, n_jobs=-1)
# подгоняем модель
forest.fit(X, y)
# вычисляем R-квадрат по методу OOB
print('R-квадрат по методу OOB: {:.2f}'.format(forest.oob_score_))

R-квадрат по методу OOB: -0.36


In [14]:
# прогнозируем значения зависимой переменной по методу OOB
oob_predictions = forest.oob_prediction_
# преобразуем прогнозы в объект DataFrame
oob_predictions = pd.DataFrame(oob_predictions, index=data.index, columns=['OOB_Predictions'])
# конкатенируем массив признаков, массив значений 
# зависимой переменной и OOB прогнозы
oob_results = pd.concat([X, y, oob_predictions], axis=1)
oob_results

,age,gender_Женский,gender_Мужской,days_of_delinquency,OOB_Predictions
0,70,0,1,18,17.800000
1,64,0,1,15,16.000000
2,69,1,0,17,16.000000
3,68,0,1,19,14.666667
4,65,1,0,15,14.600000
5,63,0,1,16,15.000000
6,55,1,0,13,14.333333
7,60,0,1,14,15.800000
8,67,0,1,16,17.750000
9,68,1,0,14,17.666667


In [15]:
# вычисляем сумму квадратов отклонений фактических значений
# зависимой переменной от ее среднего значения
TSS = ((oob_results['days_of_delinquency'] - 
        oob_results['days_of_delinquency'].mean()) ** 2).sum()
# вычисляем сумму квадратов отклонений фактических значений 
# зависимой переменной от спрогнозированных, при этом каждое 
# спрогнозированное значение – результат усреднения средних
# значений, вычисленных деревьями по OOB выборкам 
RSS = ((oob_results['days_of_delinquency'] - 
        oob_results['OOB_Predictions']) ** 2).sum()
# вычисляем R-квадрат по методу OOB
print('R-квадрат по методу OOB, вычисленный вручную: {:.2f}'.format(1 - (RSS / TSS)))

R-квадрат по методу OOB, вычисленный вручную: -0.36


In [16]:
# вычисляем R-квадрат по обычному методу
print('R-квадрат по обычному методу: {:.2f}'.format(
    forest.score(X, y)))

R-квадрат по обычному методу: 0.85


In [17]:
# прогнозируем значения зависимой переменной по обычному методу
predictions = forest.predict(X)
# преобразуем прогнозы в объект DataFrame
predictions = pd.DataFrame(predictions, index=data.index, columns=['Predictions'])
# конкатенируем массив признаков, массив значений
# зависимой переменной и прогнозы
results = pd.concat([X, y, predictions], axis=1)
results

,age,gender_Женский,gender_Мужской,days_of_delinquency,Predictions
0,70,0,1,18,17.9
1,64,0,1,15,15.3
2,69,1,0,17,16.5
3,68,0,1,19,17.7
4,65,1,0,15,14.8
5,63,0,1,16,15.9
6,55,1,0,13,13.4
7,60,0,1,14,14.9
8,67,0,1,16,16.7
9,68,1,0,14,15.1


In [18]:
# вычисляем сумму квадратов отклонений фактических значений
# зависимой переменной от ее среднего значения
TSS = ((results['days_of_delinquency'] -
        results['days_of_delinquency'].mean()) ** 2).sum()
# вычисляем сумму квадратов отклонений фактических значений 
# зависимой переменной от спрогнозированных, при этом каждое 
# спрогнозированное значение – результат усреднения средних
# значений, вычисленных деревьями по всем бутстреп-выборкам 
RSS = ((results['days_of_delinquency'] - 
        results['Predictions']) ** 2).sum()
# вычисляем R-квадрат по обычному методу
print('R-квадрат по обычному методу, вычисленный вручную: {:.2f}'.format(1 - (RSS / TSS)))

R-квадрат по обычному методу, вычисленный вручную: 0.85
